In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql import Window
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType
import datetime as dt
from pyspark.sql import functions as F
from pyspark.sql import Window
import os
import re
import datetime as dt

# Config

In [2]:
# ==========================================
# 사용자 설정 (원하는 지역과 시군구를 입력하세요)
# ==========================================
REGION = "경기도"          # "ex.서울특별시"
SIGUNGU = "평택시"    # "ex.강남구"

# 베이스 경로
OUTPUT_BASE = "/opt/spark/data/output/silver_stage_2"
TOJI_LIST_PATH = f"/opt/spark/data/output/silver_stage_1/{REGION}_{SIGUNGU.replace(' ', '_')}_filtered_final_toji"
TOJI_OWNER_PATH = "/opt/spark/data/tojidaejang/parquet/ocr_result.parquet"

In [3]:
# Spark Session 설정
spark = SparkSession.builder \
    .appName(f'silver_stage_2_{REGION}_{SIGUNGU.replace(" ", "_")}') \
    .master("spark://spark-master:7077") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .config("spark.sql.shuffle.partitions", "400") \
    .getOrCreate()

Picked up JAVA_TOOL_OPTIONS: -Dfile.encoding=UTF-8
Picked up JAVA_TOOL_OPTIONS: -Dfile.encoding=UTF-8
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/17 22:25:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# 필터링된 토지 리스트 로드

In [4]:
filtered_toji_df = spark.read.parquet(TOJI_LIST_PATH)

print("로드 row 수:", filtered_toji_df.count())
filtered_toji_df.printSchema()
filtered_toji_df.show(5, truncate=False)

로드 row 수: 8683
root
 |-- 법정동명: string (nullable = true)
 |-- 본번: string (nullable = true)
 |-- 고유번호: string (nullable = true)
 |-- 지번: string (nullable = true)
 |-- 소유권변동일자: string (nullable = true)
 |-- 토지면적: double (nullable = true)
 |-- 지목: string (nullable = true)
 |-- 부번: string (nullable = true)
 |-- 관리_건축물대장_PK: string (nullable = true)
 |-- 옥외자주식면적: string (nullable = true)
 |-- 대장_구분_코드: string (nullable = true)
 |-- 업체명: string (nullable = true)
 |-- 대표자: string (nullable = true)
 |-- 소재지_정제: string (nullable = true)

+----------------------------------+----+-------------------+------+--------------+--------+----+----+------------------+--------------+--------------+--------------------------+------+-----------------------------------------+
|법정동명                          |본번|고유번호           |지번  |소유권변동일자|토지면적|지목|부번|관리_건축물대장_PK|옥외자주식면적|대장_구분_코드|업체명                    |대표자|소재지_정제                              |
+----------------------------------+----+-------------------+------+

# 각 토지의 주차장 면적 컬럼 추가

In [5]:
# 1) 필요한 컬럼만 + 주차장면적 계산
filtered_toji_df = (
    filtered_toji_df
    .select(
        "법정동명", "본번", "부번", "고유번호", "소재지_정제", "소유권변동일자", "지목", "업체명", "대표자",
        "토지면적", "옥외자주식면적"
    )
    .withColumn(
        "옥외자주식면적_d",
        F.regexp_replace(F.col("옥외자주식면적"), ",", "").cast("double")
    )
    .withColumn(
        "주차장면적",
        F.when(
            F.col("업체명").isNotNull(),
            F.coalesce(F.col("옥외자주식면적_d"), F.lit(0.0))
        ).when(
            (F.col("업체명").isNull()) & (F.col("지목").isin("대", "잡종지", "주차장", "창고용지")),
            F.coalesce(F.col("토지면적"), F.lit(0.0))
        ).otherwise(F.lit(0.0))
    )
    .drop("토지면적", "옥외자주식면적", "옥외자주식면적_d")
)

print("로드 row 수:", filtered_toji_df.count())
filtered_toji_df.printSchema()
filtered_toji_df.show(5, truncate=False)

로드 row 수: 8683
root
 |-- 법정동명: string (nullable = true)
 |-- 본번: string (nullable = true)
 |-- 부번: string (nullable = true)
 |-- 고유번호: string (nullable = true)
 |-- 소재지_정제: string (nullable = true)
 |-- 소유권변동일자: string (nullable = true)
 |-- 지목: string (nullable = true)
 |-- 업체명: string (nullable = true)
 |-- 대표자: string (nullable = true)
 |-- 주차장면적: double (nullable = false)

+----------------------------------+----+----+-------------------+-----------------------------------------+--------------+----+--------------------------+------+----------+
|법정동명                          |본번|부번|고유번호           |소재지_정제                              |소유권변동일자|지목|업체명                    |대표자|주차장면적|
+----------------------------------+----+----+-------------------+-----------------------------------------+--------------+----+--------------------------+------+----------+
|경기도 용인시 처인구 김량장동     |333 |1   |4146110100103330001|경기도 용인시 처인구 금령로 24           |1987-06-24    |대  |보석포차                  |유*숙 |0.0  

# 토지 그룹 지주 로드

In [6]:
toji_group_df = spark.read.parquet(TOJI_OWNER_PATH).dropDuplicates(["주소", "본번", "지주"])

print("로드 row 수:", toji_group_df.count())
toji_group_df.printSchema()
toji_group_df.show(10, truncate=False)

로드 row 수: 4051
root
 |-- 주소: string (nullable = true)
 |-- 본번: string (nullable = true)
 |-- 부번: string (nullable = true)
 |-- 지주: string (nullable = true)
 |-- 소유권변동일자: string (nullable = true)

+---------------------------+----+----+------+--------------+
|주소                       |본번|부번|지주  |소유권변동일자|
+---------------------------+----+----+------+--------------+
|경기도 용인시 처인구 고림동|193 |4   |신기훈|2023-12-26    |
|경기도 용인시 처인구 고림동|193 |NULL|오제천|2025-07-14    |
|경기도 용인시 처인구 고림동|195 |1   |김영호|2010-07-15    |
|경기도 용인시 처인구 고림동|195 |4   |방지현|2025-03-10    |
|경기도 용인시 처인구 고림동|195 |7   |한다솜|2022-06-08    |
|경기도 용인시 처인구 고림동|195 |6   |한상길|1993-05-19    |
|경기도 용인시 처인구 고림동|208 |2   |김광식|2006-11-17    |
|경기도 용인시 처인구 고림동|208 |13  |최경배|2025-02-28    |
|경기도 용인시 처인구 고림동|213 |1   |송보현|2020-01-17    |
|경기도 용인시 처인구 고림동|264 |5   |선미영|2013-09-05    |
+---------------------------+----+----+------+--------------+
only showing top 10 rows


# 지주를 알고 있는 토지만 필터링 (inner join)
- 본번, 소유권변동일자가 같다면 같은 지주가 구입했다고 전제

In [7]:
t = filtered_toji_df.alias("t")
o = toji_group_df.alias("o")

toji_with_owner_df = (
    t.join(
        o,
        (F.col("t.법정동명") == F.col("o.주소")) &
        (F.col("t.본번") == F.col("o.본번")) &
        (F.col("t.소유권변동일자") == F.col("o.소유권변동일자")),
        how="inner"
    )
    .select(
        "t.*",                     
        F.col("o.지주").alias("지주"),
    )
)

print("row 수:", toji_with_owner_df.count())
toji_with_owner_df \
    .orderBy(F.col("주차장면적").desc()) \
    .show(10, truncate=False)

row 수: 6303
+----------------------------------+----+----+-------------------+------------------------------------+--------------+--------+---------------------+------+----------+------------------+
|법정동명                          |본번|부번|고유번호           |소재지_정제                         |소유권변동일자|지목    |업체명               |대표자|주차장면적|지주              |
+----------------------------------+----+----+-------------------+------------------------------------+--------------+--------+---------------------+------+----------+------------------+
|경기도 용인시 처인구 모현읍 일산리|112 |1   |4146125324101120001|NULL                                |2024-05-27    |잡종지  |NULL                 |NULL  |8404.0    |최우경            |
|경기도 용인시 처인구 남사읍 봉무리|670 |7   |4146125921106700007|NULL                                |2023-06-05    |잡종지  |NULL                 |NULL  |7343.0    |이삼식            |
|경기도 용인시 처인구 백암면 가좌리|356 |10  |4146135033103560010|NULL                                |2018-09-20    |창고용지|NULL                 |NULL

# 지주가 음식점 대표인 토지 그룹 필터링

In [8]:
def name_match(col_rep, col_owner):

    # 1️⃣ 대표자에서 공동표기 제거
    rep_clean = F.regexp_replace(
        col_rep,
        r"\s*외\s*\d*\s*[인명]?",   # 외 1인 / 외2명 / 외 / 외3
        ""
    )

    # 2️⃣ * 제거
    rep_clean = F.regexp_replace(rep_clean, r"\*", "")

    # 3️⃣ 비교
    return (
        col_rep.isNotNull() &
        col_owner.isNotNull() &
        (F.length(rep_clean) >= 2) &
        (F.substring(rep_clean, 1, 1) == F.substring(col_owner, 1, 1)) &
        (
            F.substring(rep_clean, F.length(rep_clean), 1) ==
            F.substring(col_owner, F.length(col_owner), 1)
        )
    )

df = toji_with_owner_df.withColumn(
    "is_match",
    name_match(F.col("대표자"), F.col("지주")).cast("int")
)

filtered_df = df.join(
    df.groupBy("법정동명","본번","지주")
      .agg(F.max("is_match").alias("has_match"))
      .filter("has_match=1"),
    ["법정동명","본번","지주"]
).drop("is_match","has_match")

print(filtered_df.count())
filtered_df.show(100, truncate=False)

239
+----------------------------------+----+------+----+-------------------+------------------------------------------+--------------+----+-------------------------------+------------+----------+
|법정동명                          |본번|지주  |부번|고유번호           |소재지_정제                               |소유권변동일자|지목|업체명                         |대표자      |주차장면적|
+----------------------------------+----+------+----+-------------------+------------------------------------------+--------------+----+-------------------------------+------------+----------+
|경기도 용인시 처인구 고림동       |208 |최경배|12  |4146110600102080012|NULL                                      |2025-02-28    |도로|NULL                           |NULL        |0.0       |
|경기도 용인시 처인구 고림동       |208 |최경배|13  |4146110600102080013|NULL                                      |2025-02-28    |전  |NULL                           |NULL        |0.0       |
|경기도 용인시 처인구 고림동       |208 |최경배|11  |4146110600102080011|경기도 용인시 처인구 한터로 233-2         |2025-02-28    

# 토지 그룹별 주차장 면적 계산

In [9]:
parking_max_df = (
    filtered_df
    .groupBy("법정동명", "본번", "지주", "부번")
    .agg(F.max("주차장면적").alias("주차장면적_max"))
)

parking_sum_df = (
    parking_max_df
    .groupBy("법정동명", "본번", "지주")
    .agg(
        F.sum("주차장면적_max").alias("유휴부지면적")
    )
)

print(parking_sum_df.count())
parking_sum_df.show(10, truncate=False)

124
+-----------------------------+----+------+------------+
|법정동명                     |본번|지주  |유휴부지면적|
+-----------------------------+----+------+------------+
|경기도 용인시 처인구 고림동  |208 |최경배|12.5        |
|경기도 용인시 처인구 고림동  |341 |김성현|413.82      |
|경기도 용인시 처인구 고림동  |440 |박주희|23.0        |
|경기도 용인시 처인구 고림동  |460 |김명선|25.0        |
|경기도 용인시 처인구 고림동  |599 |박용성|46.0        |
|경기도 용인시 처인구 고림동  |747 |박병현|11.5        |
|경기도 용인시 처인구 고림동  |820 |안성자|0.0         |
|경기도 용인시 처인구 고림동  |966 |이현수|112.0       |
|경기도 용인시 처인구 김량장동|133 |김선영|0.0         |
|경기도 용인시 처인구 김량장동|133 |김성자|0.0         |
+-----------------------------+----+------+------------+
only showing top 10 rows


# 토지 그룹별 업체 정보 추가

In [10]:
match_df = toji_with_owner_df.filter(
    name_match(F.col("대표자"), F.col("지주"))
)

final_df = (
    match_df.alias("r")
    .join(
        parking_sum_df.alias("p"),
        on=["법정동명", "본번", "지주"],
        how="left"
    )
    .withColumnRenamed("주차장면적", "min_유휴부지면적")
    .withColumnRenamed("소재지_정제", "도로명주소")
    .withColumnRenamed("고유번호", "PNU코드")
    .drop("소유권변동일자", "지목")
)

final_df = (
    final_df
    .withColumn(
        "min_유휴부지면적_d",
        F.regexp_replace(F.col("min_유휴부지면적").cast("string"), ",", "").cast("double")
    )
    .withColumn(
        "유휴부지면적_d",
        F.col("유휴부지면적").cast("double")
    )
    .withColumn(
        "분모",
        F.greatest(
            F.coalesce(F.col("min_유휴부지면적_d"), F.lit(0.0)),
            F.coalesce(F.col("유휴부지면적_d"), F.lit(0.0))
        )
    )
    .withColumn(
        "신뢰도점수",
        F.when(
            F.col("min_유휴부지면적_d").isNull() | F.col("유휴부지면적_d").isNull(),
            F.lit(0.0)
        ).when(
            F.col("분모") == 0,
            F.lit(1.0)
        ).otherwise(
            # 1 - 상대오차
            F.lit(1.0) - (F.abs(F.col("유휴부지면적_d") - F.col("min_유휴부지면적_d")) / F.col("분모"))
        )
    )
    # 0~1 클리핑
    .withColumn("신뢰도점수", F.when(F.col("신뢰도점수") < 0, 0.0)
                           .when(F.col("신뢰도점수") > 1, 1.0)
                           .otherwise(F.col("신뢰도점수")))
    .withColumn("신뢰도점수", F.round(F.col("신뢰도점수"), 2))
    # 임시 컬럼 정리
    .drop("min_유휴부지면적_d", "유휴부지면적_d", "분모")
)

print(final_df.count())
final_df \
    .orderBy(F.col("유휴부지면적").desc()) \
    .show(20, truncate=False)


124
+----------------------------------+----+------+----+-------------------+-------------------------------------------+--------------------+------------+----------------+------------+----------+
|법정동명                          |본번|지주  |부번|PNU코드            |도로명주소                                 |업체명              |대표자      |min_유휴부지면적|유휴부지면적|신뢰도점수|
+----------------------------------+----+------+----+-------------------+-------------------------------------------+--------------------+------------+----------------+------------+----------+
|경기도 용인시 처인구 포곡읍 둔전리|155 |이영호|15  |4146125023101550015|경기도 용인시 처인구 포곡로124번길 12      |한끼국밥(둔전점)    |이*호       |40.0            |904.0       |0.04      |
|경기도 용인시 처인구 유방동       |759 |김영일|12  |4146110500107590012|경기도 용인시 처인구 백령로 56             |석성산              |김*일       |294.4           |846.4       |0.35      |
|경기도 용인시 처인구 모현읍 능원리|372 |최종인|1   |4146125327103720001|경기도 용인시 처인구 능원로132번길 2       |몽뻬르베이커리카페  |최*인 외 1명|75.0            |840.0       |0.09    

In [11]:
FINAL_PATH = os.path.join(
    OUTPUT_BASE,
    f"REGION={REGION}/{SIGUNGU.replace(' ', '_')}_final"
)

final_df.write \
    .mode("overwrite") \
    .parquet(FINAL_PATH)

print("✅ final_df 저장 완료:", FINAL_PATH)


✅ final_df 저장 완료: /opt/spark/data/output/silver_stage_2/REGION=경기도/용인시_처인구_final


In [12]:
spark.stop()